In [2]:
from multiprocessing.sharedctypes import Value
from tkinter import *
from tkinter import ttk
import time
import multiprocessing as mp
from multiprocessing import Process 
import threading
import random
import queue
import asyncio
import math

In [3]:
def is_even(num):
    if num%2==0:
        return True
    return False

def parallel_process(min_val, max_val, result):
    r = []
    while(min_val <= max_val):
        if(is_even(min_val)):
            r.append(min_val)
        min_val += 1
        
    result.value += len(r)
        
def parallel_command(min_val, max_val):
    tasks = []
    result = mp.Value("i", 0)
    start = time.perf_counter()
    i = 0
    cpu_count = mp.cpu_count()
    max_val = max_val + 1
    diff = max_val - min_val
    size = math.ceil(diff / cpu_count)
    while(i < cpu_count):
        s = min_val + (size * i)
        e = s + size
        if(e > max_val):
            e = max_val
        p = Process(target=parallel_process(s, e - 1, result))
        p.start()
        tasks.append(p)
        i += 1
    while(len(tasks)):
        tasks.pop().join()
    stop = time.perf_counter()
    root.results.insert(1, "Par. from " + str(int(root.min_entry.get())) + " to " + str(int(root.max_entry.get())) + ": " + str(result.value) + " took: " + str(stop - start))
    
def gui_init(master, sequential_command, parallel_command):
    # Set up the GUI
    results_frame = ttk.Frame(root)
    results_frame.grid()
    results = Listbox(results_frame, width=50)
    results.grid(row=0)
    master.results = results
    root_frame = ttk.Frame(root)
    root_frame.grid(rowspan=2)
    controls_frame = ttk.Frame(root_frame)
    controls_frame.grid(row=2)
    quit_button = ttk.Button(controls_frame, text="Quit!", command=master.destroy)
    quit_button.pack(side="right")
    sequential_button = ttk.Button(controls_frame, text="Sequential", command=sequential_command)
    sequential_button.pack(side="right")
    parallel_button = ttk.Button(controls_frame, text="Parallel", command=parallel_command)
    inputs_frame = ttk.Frame(root_frame)
    inputs_frame.grid(row=1)
    label_min = Label(inputs_frame, text="Min.")
    label_min.pack(side="left")
    min_entry = Entry(inputs_frame)
    min_entry.pack(side="left")
    max_entry = Entry(inputs_frame)
    max_entry.pack(side="right")
    master.max_entry =max_entry
    master.min_entry =min_entry
    label_max = Label(inputs_frame, text="Max.")
    label_max.pack(side="right")
    parallel_button.pack(side="right")

def periodicCall():
    root.after(200, periodicCall)

def sequential_click():
    thread1 = threading.Thread(target=sequential_command)
    thread1.start()
    #periodicCall()

def sequential_command():
    result = []
    min_val = int(root.min_entry.get())
    max_val = int(root.max_entry.get())
    start = time.perf_counter()
    while(min_val <= max_val):
        if(is_even(min_val)):
            result.append(min_val)
        min_val += 1
    stop = time.perf_counter()
    if(min_val > max_val):
        min_val = max_val
    root.results.insert(1, "Seq. from " + str(int(root.min_entry.get())) + " to " + str(min_val) + ": " + str(len(result)) + " took: " + str(stop - start))

def parallel_click():    
    thread1 = threading.Thread(target=parallel_command, args=(int(root.min_entry.get()), int(root.max_entry.get())))
    thread1.start(  )
    #parallel_command(int(root.min_entry.get()), int(root.max_entry.get()))
    #periodicCall()
    
if __name__ == '__main__':    
    rand = random.Random()
    lock = mp.Lock()
    root = Tk()
    queue = queue.Queue(  )
    running = 0
    # Set up the GUI part
    gui = gui_init(root, sequential_click, parallel_click)
    root.mainloop()